## Create Histograms with parameteric bin contents

Enable visualisation with JS and import ROOT

In [ ]:
import ROOT
%jsroot on

Create workspace as input to combine

In [ ]:
fOut = ROOT.TFile("param_ws.root","RECREATE")
wspace = ROOT.RooWorkspace("wspace","wspace")
wspace_import = getattr(wspace,"import")

ROOT.gSystem.Load("/cvmfs/sft.cern.ch/lcg/releases/LCG_85swan3/vdt/0.3.6/x86_64-slc6-gcc49-opt/lib/libvdt.so")
ROOT.gSystem.Load("../../lib/libHiggsAnalysisCombinedLimit.so")

 Make the RooRealVar variable as the observable for the search

In [ ]:
met = ROOT.RooRealVar("met","E_{T}^{miss}",200,1000)
plot = met.frame()

# Signal region

Make a data set for our signal region and import that into the workspace

In [ ]:
data_th1 = ROOT.TH1F("data_obs_SR","Data observed in signal region",4,200,1000)
data_th1.SetBinContent(1,100)
data_th1.SetBinContent(2,50)
data_th1.SetBinContent(3,25)
data_th1.SetBinContent(4,10)
data_hist = ROOT.RooDataHist("data_obs_SR","Data observed",ROOT.RooArgList(met),data_th1)
wspace_import(data_hist)
data_hist.plotOn(plot)

In [ ]:
c = ROOT.TCanvas()
plot.Draw()
c.Draw()

Now create the parametric binned template for the background 

Each bin will have its own parameter representing the rate in that bin

In [ ]:
bin1 = ROOT.RooRealVar("bkg_SR_bin1","Background yield in signal region, bin 1",100,0,500)
bin2 = ROOT.RooRealVar("bkg_SR_bin2","Background yield in signal region, bin 2",50,0,500)
bin3 = ROOT.RooRealVar("bkg_SR_bin3","Background yield in signal region, bin 3",25,0,500)
bin4 = ROOT.RooRealVar("bkg_SR_bin4","Background yield in signal region, bin 4",10,0,500)

bkg_SR_bins = ROOT.RooArgList()
bkg_SR_bins.add(bin1)
bkg_SR_bins.add(bin2)
bkg_SR_bins.add(bin3)
bkg_SR_bins.add(bin4)

p_bkg = ROOT.RooParametricHist("bkg_SR", "Background PDF in signal region",met,bkg_SR_bins,data_th1)

If we fit this to the data, we should get back the number of data in each bins as the MLEs 

In [ ]:
p_bkg.fitTo(data_hist)
p_bkg.plotOn(plot)
c2 = ROOT.TCanvas()
plot.Draw()
c2.Draw()

# Control region

In [ ]:
data_CRth1 = ROOT.TH1F("data_obs_CR","Data observed in control region",4,200,1000)
data_CRth1.SetBinContent(1,200)
data_CRth1.SetBinContent(2,100)
data_CRth1.SetBinContent(3,50)
data_CRth1.SetBinContent(4,20)
data_CRhist = ROOT.RooDataHist("data_obs_CR","Data observed",ROOT.RooArgList(met),data_CRth1)
wspace_import(data_CRhist)

This time the parameters for the histogram in the control region will be functions of those in the signal region

In [ ]:
TF = ROOT.RooRealVar("TF","Transfer factor",2) 
TF.setConstant() 

me = ROOT.RooRealVar("muon_eff","Muon efficiency",0,-4,4)

CRbin1 = ROOT.RooFormulaVar("bkg_CR_bin1","Background yield in CR, bin 1","@0*@1*(1+0.005*@2)",ROOT.RooArgList(TF,bin1,me))
CRbin2 = ROOT.RooFormulaVar("bkg_CR_bin2","Background yield in CR, bin 2","@0*@1*(1+0.01*@2)",ROOT.RooArgList(TF,bin2,me))
CRbin3 = ROOT.RooFormulaVar("bkg_CR_bin3","Background yield in CR, bin 3","@0*@1*(1+0.02*@2)",ROOT.RooArgList(TF,bin3,me))
CRbin4 = ROOT.RooFormulaVar("bkg_CR_bin4","Background yield in CR, bin 4","@0*@1*(1+0.04*@2)",ROOT.RooArgList(TF,bin4,me))
  
bkg_CR_bins = ROOT.RooArgList()
bkg_CR_bins.add(CRbin1)
bkg_CR_bins.add(CRbin2)
bkg_CR_bins.add(CRbin3)
bkg_CR_bins.add(CRbin4)

p_CRbkg = ROOT.RooParametricHist("bkg_CR", "Background PDF in control region",met,bkg_CR_bins,data_th1)

Make the usual `_norm` parameters for the two RooParametricHists, this can just be the sum of bins

In [ ]:
p_bkg_norm = ROOT.RooAddition("bkg_SR_norm","Total Number of events from background in signal region",bkg_SR_bins)
p_CRbkg_norm = ROOT.RooAddition("bkg_CR_norm","Total Number of events from background in control region",bkg_CR_bins)

wspace_import(p_bkg)
wspace_import(p_bkg_norm)

wspace_import(p_CRbkg)
wspace_import(p_CRbkg_norm,ROOT.RooFit.RecycleConflictNodes())

Finally, create a signal histogram template

In [ ]:
signal_th1 = ROOT.TH1F("signal_SR","Signal expected in signal region",4,200,1000)
signal_th1.SetBinContent(1,1)
signal_th1.SetBinContent(2,2)
signal_th1.SetBinContent(3,3)
signal_th1.SetBinContent(4,8)
signal_hist = ROOT.RooDataHist("signal","Data observed",ROOT.RooArgList(met),signal_th1)
wspace_import(signal_hist)

In [ ]:
wspace.Print("V")

In [ ]:
fOut.WriteTObject(wspace)
fOut.Close()